# Emplear Regresión Por Maquina de Soporte de Vectores

## Traer los datos

Obtener los datos de las viviendas.

In [41]:
import os
import pandas as pd

def load_housing_data():
    csv_path = os.path.join("datasets", "housing", "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()

In [42]:
housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


## Obtener los conjuntos de training y testing

Generar categorias para el ingreso medio

In [43]:
import numpy as np

housing["income_cat"] = pd.cut(
    housing["median_income"],
    bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
    labels=[1, 2, 3, 4, 5])

In [44]:
housing["income_cat"].head()

0    5
1    5
2    5
3    4
4    3
Name: income_cat, dtype: category
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

Dividir el conjunto de datos en dos subconjuntos, uno para entrenamiento y otro para pruebas. Ambos de forma estratificada en base a un atributo importante (que las categorias de ingresos sean proporcionales).

In [45]:
from sklearn.model_selection import StratifiedShuffleSplit

# Dividir los dos conjuntos proporcionalmente a la categoria de ingresos
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# Eliminar la columna de categoria, ya no es necesaria
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

Reasignar a "housing" los datos de entrenamiento.

In [46]:
housing = strat_train_set.drop("median_house_value", axis=1)

## Aplicar transformadores para limpieza y desarrollo

In [47]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
housing_num = housing.drop("ocean_proximity", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
])
housing_prepared = full_pipeline.fit_transform(housing)

## Entrenamiento

El mejor rendimiento (el error más bajo) hasta el momento se logró son RandomForestRegressor. La siguiente es una tabla de RMSE.
|    LinealReg   |     TreeReg    |  RandomForest  |
|     :----:     |     :----:     |    :----:      |
|     68627.87   |     71035.40   |    47293.78    |

Emplear SVMR para mejorar el rendimiento y bajar el nivel de error. Primero entrenar el modelo usando GridSearch.

In [75]:
from sklearn.svm import SVR

# lin_reg = LinearRegression()
# lin_reg.fit(housing_prepared, housing_labels)

# housing_predictions = lin_reg.predict(housing_prepared)
# lin_mse = mean_squared_error(housing_labels, housing_predictions)
# lin_rmse = np.sqrt(lin_mse)
# lin_rmse

svm_reg = SVR()
# svm_reg.fit(housing_prepared, housing_labels)



from sklearn.model_selection import GridSearchCV

param_grid = [
    {'kernel': ['linear'], 'C': [80, 90, 100]},
    # {'kernel': ['rbf'], 'C': [1, 2, 3], 'gamma': [1, 2, 3]},
]

grid_search = GridSearchCV(
    svm_reg,
    param_grid,
    cv=2,
    scoring='neg_mean_squared_error',
    return_train_score=True
    )
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=2, estimator=SVR(),
             param_grid=[{'C': [80, 90, 100], 'kernel': ['linear']}],
             return_train_score=True, scoring='neg_mean_squared_error')

Conocer las combinaciones ejecutadas

In [76]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

73295.10004930786 {'C': 80, 'kernel': 'linear'}
72911.635694381 {'C': 90, 'kernel': 'linear'}
72615.79063907597 {'C': 100, 'kernel': 'linear'}


Conocer el mejor estimador

In [77]:
grid_search.best_estimator_

SVR(C=100, kernel='linear')

Calcular el nivel de RMSE con el mejor estimador.

In [78]:
from sklearn.metrics import mean_squared_error

housing_predictions = grid_search.best_estimator_.predict(housing_prepared)
svm_mse = mean_squared_error(housing_labels, housing_predictions)
svm_mse = np.sqrt(svm_mse)
svm_mse

71258.59821420867